In [1]:
import requests 
from dotenv import load_dotenv
import os
import json
from bs4 import BeautifulSoup
from IPython.display import clear_output
import dill 

In [23]:
load_dotenv()

url_get_food = "https://api.spoonacular.com/recipes/random"
url_get_nutition = "https://api.spoonacular.com/recipes/{id}/nutritionWidget.json"
url_post_food = "http://127.0.0.1:8000/food/"
api_key = os.getenv("apiKey")

In [15]:
def get_random_food(api_key, number_of_recipes=100):
    params = {"apiKey": api_key, "number": number_of_recipes}
    response = requests.get(url_get_food, params=params)
    return response.json()

def get_nutritions(api_key, id):
    params = {"apiKey": api_key}
    response = requests.get(url_get_nutition.format(id=id), params=params)
    return response.json()

In [53]:
def create_recipe(name, description, image, nutrients, instructions, ingredients):
    
    header = {"Content-Type": "application/json"}
    
    data = {
        "name": name,
        "description": description,
        "image": image,
        "nutrients": nutrients,
        "instructions": instructions,
        "ingredients": ingredients
    }
    response = requests.post(url_post_food, headers=header, data=json.dumps(data))
    if response.status_code == 201:
        return response.json()
    return response.json()

In [3]:
def handle_nutrient(recipe_id):
    nutrient = get_nutritions(api_key, recipe_id)
    return nutrient.get("nutrients")

def handle_ingredients(extendedIngredients):
    res = []
    for ingredient in extendedIngredients:
        res.append({ "original" : ingredient.get("original")})
    return res 

def handle_instructions(analyzedInstructions):
    res = []
    steps = analyzedInstructions.get("steps")
    if not steps:
        return res 
    for instruction in steps:
        res.append({"number": instruction.get("number"), "step": instruction.get("step")})
    return res

def handle_description(summary):
    soup = BeautifulSoup(summary, 'html.parser')
    return soup.get_text()

In [35]:
data = get_random_food(api_key)

In [17]:
def handle_recipe(recipe):
    name = recipe.get("title")
    description = handle_description(recipe.get("summary"))
    image = recipe.get("image")
    nutrients = handle_nutrient(recipe.get("id"))
    instructions = handle_instructions(recipe.get("analyzedInstructions")[0])
    ingredients = handle_ingredients(recipe.get("extendedIngredients"))
    if name and description and image and nutrients and instructions and ingredients:
        return create_recipe(name, description, image, nutrients, instructions, ingredients)
    return None
    

In [65]:
count = 0

for recipe in data.get("recipes"):
    handle_recipe(recipe)
    clear_output(wait=True)
    print(count/len(data.get("recipes")))
    count += 1
    

0.99


In [62]:
print(handle_description(recipe.get("summary")))

If you want to add more gluten free and lacto ovo vegetarian recipes to your recipe box, No-Bake Chocolate Peanut Butter Macaroons might be a recipe you should try. This recipe makes 1 servings with 2094 calories, 31g of protein, and 88g of fat each. For $2.27 per serving, this recipe covers 41% of your daily requirements of vitamins and minerals. It works best as a dessert, and is done in approximately 45 minutes. This recipe from Foodista has 2 fans. Head to the store and pick up milk, peanut butter to garnish, cocoa, and a few other things to make it today. All things considered, we decided this recipe deserves a spoonacular score of 73%. This score is solid. Similar recipes include Chocolate Peanut Butter Macaroons, Chocolate Peanut Butter Burgers (French Macaroons), and Peanut Butter Macaroons.


In [2]:
import requests
import json
url = "http://127.0.0.1:8000/food/{id}/"

for i in range(5, 100):
    response = requests.get(url.format(id=i))
    if response.status_code == 200:
        res_json = response.json()
        calories = list(filter(lambda x : x.get("name") == "Calories", res_json.get("nutrients")))
        if calories:
            calories = int(calories[0].get("amount"))
            response2 = requests.patch(url.format(id=i), data=json.dumps({"calories": calories}), headers={"Content-Type": "application/json"})
            if not response2.status_code == 200:
                print(res_json)


In [47]:
create_recipe("test", "test", "test", [], [], [])

{'detail': 'Unsupported media type "text/plain" in request.'}

In [67]:
dill.dump_session("sesson.pkl")

In [ ]:
dill.load_session("sesson.pkl")